## 네이버 영화 감상평 댓글 감정 분석
---

#### 프로젝트에 사용된 데이터 소개
- Naver sentiment movie corpus deta를 사용합니다.
- 데이터 출처: https://github.com/e9t/nsmc

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 데이터 다운 및 기본 처리
- 전처리 전에 데이터의 형태와 내용을 확인해보는 과정입니다.

In [ ]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

In [ ]:
!ls

In [ ]:
!cat ratings_train.txt | head -n10

In [ ]:
train_df = pd.read_csv("ratings_train.txt", sep = "\t")
test_df = pd.read_csv("ratings_test.txt", sep = "\t")

In [ ]:
train_df

In [ ]:
# data size
print("trian data size: ", train_df.shape)
print("test data size: ", test_df.shape)

In [ ]:
# data type
print(train_df.dtypes, "\n")
print(test_df.dtypes)

In [ ]:
# 댓글 길이 분포 확인
plt.bar(range(2000), [len(str(text)) for text in train_df.sample(2000).document])
plt.show()

#### 전처리
- KoNLPy의 Okt를 사용 -> 어디다 쓰는거지?
- Sentencepiece를 사용

In [ ]:
# 형태소 분석기 설치
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# sentencepiece 설치
!pip install sentencepiece

In [ ]:
import sentencepiece as spm

In [ ]:
# document값만 뽑아내기 (tokenizer를 학습시키기 위함)
full_document = np.concatenate([train_df.document.values, test_df.document.values])

In [ ]:
full_document

In [ ]:
# 모은 document값들을 txt로 바꾸고 정리
with open('./full_document.txt', 'w', encoding="utf-8") as f:
    for line in full_document:
        if len(str(line).strip()) == 0:  continue
        try:
            f.write(line + "\n")
        except: continue

In [ ]:
!cat full_document.txt | head -n10

In [ ]:
# Tokenizer 학습
spm.SentencePieceTrainer.train('--input=full_document.txt --model_prefix=m --vocab_size=20000')

In [ ]:
# model: 실제로 사용되는 tokenizer 모델
# vocab: 참조하는 단어 집합
!ls -alh

In [ ]:
# vocab에 대한 정보 확인
with open('./m.vocab', encoding='utf-8') as f:
    Vo = [doc.strip().split("\t") for doc in f]

word2idx = {w[0]: i for i, w in enumerate(Vo)}
word2idx

In [ ]:
# Tokenizer 모델 불러오기
sp = spm.SentencePieceProcessor()
sp.load('m.model')

In [ ]:
# string으로 tokenize
sp.encode_as_pieces('영화 보고싶다!')

In [ ]:
# ids로 tokenize -> m.vocab에서 token에 붙은 id와 동일 한 것을 확인
sp.encode_as_ids('영화 보고싶다!')

In [ ]:
# string으로 decoding
sp.decode_pieces(['▁영화', '▁보고싶다', '!'])

In [ ]:
# ids로 decoding
sp.decode_ids([7, 1847, 20])

In [ ]:
train_df

In [ ]:
# BoW: Back of Words
# Data frame에 bow 필드 추가
train_df['bow'] = train_df.document.apply(lambda x: sp.encode_as_ids(str(x)))
test_df['bow'] = test_df.document.apply(lambda x: sp.encode_as_ids(str(x)))

In [ ]:
train_df

In [ ]:
# data frame으로부터 bow 필드 빼내기
train_text = train_df.bow.values
test_text = test_df.bow.values

In [ ]:
# data frame으로부터 label값 빼내기
train_sentiment = train_df.label.values
test_sentiment = test_df.label.values

In [ ]:
# bow 필드를 실제 bow값으로 바꿔주기 (pad 추가)
train_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_text, value=0)
test_bow_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, value=0)

In [ ]:
train_bow_text

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_bow_text.shape, test_bow_text.shape

In [ ]:
# 단어들의 빈도수 알아보기
import collections
word_count = collections.Counter()

for text in train_text:
    word_count.update(text)
for text in test_text:
    word_count.update(text)

In [ ]:
word_count.most_common(10)

In [ ]:
# ids로 decoding
sp.decode_ids([7])

In [ ]:
# n번 이하로 나온 word를 삭제하는 함수
def cut_by_count(texts, n):
    return np.array([[word for word in text if word_count[word]>=n]for text in texts])

In [ ]:
train_cut_text = cut_by_count(train_text, 20)
test_cut_text = cut_by_count(test_text, 20)

In [ ]:
train_cut_text.shape, test_cut_text.shape

In [ ]:
# 댓글 길이를 최대 100으로 하여 bow 생성
train_cut_bow_text2 = tf.keras.preprocessing.sequence.pad_sequences(train_text, value=0, maxlen=100)
test_cut_bow_text2 = tf.keras.preprocessing.sequence.pad_sequences(test_text, value=0, maxlen=100)

In [ ]:
train_cut_bow_text2.shape, test_cut_bow_text2.shape

In [ ]:
# sentiment를 one-hot encoding으로 변경
train_onehot_sentiment = keras.utils.to_categorical(train_sentiment)
test_onehot_sentiment = keras.utils.to_categorical(test_sentiment)

In [ ]:
train_onehot_sentiment.shape, test_onehot_sentiment.shape

In [ ]:
# sentiment 인덱스를 긍,부정 text label로 변환하는 함수
raw_labels = ['bad', 'good']

def sentiment2label(idx):
    return raw_labels[idx]

sentiment2label(0), sentiment2label(1)

#### 시각화

#### 모델링 및 학습

In [ ]:
# seq2seq 모델 만들기

from keras.layers import Input, Embedding, GRU, Dense
from keras.models import Model

def Seq2Seq():
    inputs_x_bow = Input(shape=(100,))
    embedding = Embedding(20000, 120)
    x = embedding(inputs_x_bow)
    z = GRU(64)(x)
    y = Dense(2, activation='softmax')(z)

    model = Model(inputs_x_bow, y)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = Seq2Seq()

In [ ]:
model.summary()

In [ ]:
# 모델 학습시키기
hist = model.fit(train_cut_bow_text2, train_onehot_sentiment,
                 validation_data = (test_cut_bow_text2, test_onehot_sentiment),
                 verbose=1, epochs=2)

In [ ]:
# text를 bow로 변환해주는 함수
def text2bow(text, maxlen=150):
    seq = sp.encode_as_ids(text)
    bow = tf.keras.preprocessing.sequence.pad_sequences([seq], value=0, maxlen=maxlen)
    return bow

In [ ]:
# 학습 결과 및 성능 시각화
plt.plot(hist.history['accuracy'], label = 'accuracy')
plt.plot(hist.history['val_accuracy'], label = 'val_accuracy')
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label = 'val_loss')
plt.legend(loc='upper left')
plt.show()

#### 학습 결과 확인 및 모델 저장

In [ ]:
model.predict(text2bow('이 영화 꼭 보세요')[...,-100:])

In [ ]:
model.predict(text2bow('이 영화 보지 마세요ㅠㅠ')[...,-100:])

In [ ]:
model.predict(text2bow('너무 재밌어;;')[...,-100:])

In [ ]:
model.predict(text2bow('너무 재밌어!!')[...,-100:])